In [1]:
import pandas as pd 
from scipy import spatial
import datetime as dt
import numpy as np

In [2]:
df = pd.read_csv("BigBasketDataset.tsv", sep="\t")

In [3]:
df.head()

,Member,Order,SKU,Created On,Description
0,M09736,6468572,34993740,22-09-2014 22:45,Other Sauces
1,M09736,6468572,15669800,22-09-2014 22:45,Cashews
2,M09736,6468572,34989501,22-09-2014 22:45,Other Dals
3,M09736,6468572,7572303,22-09-2014 22:45,Namkeen
4,M09736,6468572,15669856,22-09-2014 22:45,Sugar


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62141 entries, 0 to 62140
Data columns (total 5 columns):
Member         62141 non-null object
Order          62141 non-null int64
SKU            62141 non-null int64
Created On     62141 non-null object
Description    62141 non-null object
dtypes: int64(2), object(3)
memory usage: 2.4+ MB


In [5]:
df.rename(columns={"Created On": "Date"}, inplace=True)

In [6]:
df.Date = pd.to_datetime(df.Date)

In [7]:
print("Number of customers:", len(df.Member.unique()))
print("Number of Orders:", len(df.Order.unique()))
print("Number of Product Designations:", len(df.Description.unique()))
print("Number of SKU:", len(df.SKU.unique()))
print("First Order:", df.Date.min())
print("Last Order:", df.Date.max())

Number of customers: 106
Number of Orders: 8387
Number of Product Designations: 216
Number of SKU: 1732
First Order: 2011-12-03 10:18:00
Last Order: 2014-10-06 14:21:00


In [8]:
import plotly.graph_objects as go
x = pd.DataFrame(df[["Member", "Order"]].groupby("Member").agg({"Order": "nunique"})).Order

fig = go.Figure(data=[go.Histogram(x=x, xbins=dict(size=10))])
fig.update_layout(title_text="Number of orders by customers", plot_bgcolor="white", showlegend=False)

fig.show()

In [9]:
x = pd.DataFrame(df[["SKU", "Order"]].groupby("Order").agg({"SKU": "nunique"})).SKU

fig = go.Figure(data=[go.Histogram(x=x, xbins=dict(size=1))])
fig.update_layout(title_text="Order size: Number of items per order", plot_bgcolor="white", showlegend=False)

fig.show()

In [10]:
bestselling_products = pd.DataFrame(df.Description.value_counts())
bestselling_products_bins = ["0-50" if i <= 50 else "51-200" if i <=200 else "201-500" if i <= 500 else "501-1000" if i <= 1000 else "1000+" for i in bestselling_products.Description.values]

fig = go.Figure(data=[go.Histogram(x=bestselling_products_bins)])
fig.update_layout(title_text="Number of items per order", plot_bgcolor="white", showlegend=False)

fig.show()

In [11]:
import plotly.express as px

fig = px.bar(y=bestselling_products.Description.values, x=bestselling_products.index)
fig.update_layout(title_text="Bestselling products", plot_bgcolor="white", showlegend=False)
fig.show()

In [12]:
# Order over time 
df["Year"] = df.Date.dt.year
y = df[["Year", "Order"]].groupby("Year").agg({"Order": "nunique"}).Order
fig = go.Figure(data=go.Scatter(x=y.index[1:], y=y[1:]))
fig.update_layout(title_text="Orders per Year", plot_bgcolor="white", showlegend=False)
fig.show()

df['MonthYear'] = pd.to_datetime(df['Date']).dt.to_period('M')
y = df[df.MonthYear != "2014-10"][["MonthYear", "Order"]].groupby("MonthYear").agg({"Order": "nunique"}).Order
fig = go.Figure(data=go.Scatter(x=y.index.astype(str), y=y))
fig.update_layout(title_text="Orders development over last year", plot_bgcolor="white", showlegend=False)
fig.show()

df["DayOfWeek"] = df.Date.dt.day_name()
y = df[["DayOfWeek", "Order"]].groupby("DayOfWeek").agg({"Order": "nunique"}).Order.reindex(["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"])
fig = go.Figure(data=go.Scatter(x=y.index.astype(str), y=y))
fig.update_layout(title_text="Order per Weekday", plot_bgcolor="white", showlegend=False)
fig.show()

In [13]:
y= pd.DataFrame(df[df.MonthYear != "2014-10"][["SKU", "Order", "MonthYear"]].groupby(["MonthYear", "Order"]).count().SKU).reset_index()[["MonthYear", "SKU"]].groupby(["MonthYear"]).mean()

fig = go.Figure(data=go.Scatter(x=y.index.astype(str), y=y.SKU))
fig.update_layout(title_text="Development of Order Size", plot_bgcolor="white", showlegend=False)
fig.show()

y= pd.DataFrame(df[["SKU", "Order", "DayOfWeek"]].groupby(["DayOfWeek", "Order"]).count().SKU).reset_index()[["DayOfWeek", "SKU"]].groupby(["DayOfWeek"]).mean().reindex(["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"])

fig = go.Figure(data=go.Scatter(x=y.index.astype(str), y=y.SKU))
fig.update_layout(title_text="Order size per Weekday", plot_bgcolor="white", showlegend=False)
fig.show()

In [14]:
df.Description.value_counts()[df.Description.value_counts() < 10]

Other Jams                      9
Soups                           9
Flakes                          9
Paneer & Tofu                   9
Disposable Plates & Utensils    8
                               ..
Tea                             1
Talc                            1
Cookery Books                   1
Biscuits                        1
Festive Gift Pack               1
Name: Description, Length: 76, dtype: int64

In [15]:
df["HelpCount"] = 1
bestselling_products_weekday = pd.DataFrame(df[["DayOfWeek", "Description", "HelpCount"]].groupby(["DayOfWeek", "Description"]).count()).reset_index()

In [16]:
fig = go.Figure(data=[go.Bar(x=day.Description[:15], y=day.HelpCount[:15] / day.HelpCount.sum(), name=day.DayOfWeek.values[0])
                        for day in 
                        [bestselling_products_weekday.sort_values("HelpCount",ascending=False)[bestselling_products_weekday.DayOfWeek                           == weekday]
                        for weekday in 
                        ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]]],           
                       )
fig.update_layout(title_text="Bestselling products by Weekday (in % of total)", plot_bgcolor="white", barmode='group', xaxis_tickangle=-45, width=1200,
    height=600,)
fig.show()

In [17]:
fig = go.Figure(data=[go.Bar(x=day.Description[:10], y=day.HelpCount[:10])
                        for day in 
                        [bestselling_products_weekday[bestselling_products_weekday.DayOfWeek == weekday].sort_values("HelpCount",                               ascending=False)
                        for weekday in 
                        ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]]],           
                       )

## RFQ 

In [18]:
df_rfq=df.copy()
df_rfq["Date"]=pd.to_datetime(df_rfq["Date"])
df_rfq["Date"]=pd.DatetimeIndex(df_rfq["Date"])
#Recency: Baseline data (last date in the dataset)
basedate=df_rfq.Date.max()
recency_df = df_rfq.groupby(by='Member', as_index=False)['Date'].max()
recency_df.columns = ['Member','LastOrder']
recency_df['Recency'] = recency_df['LastOrder'].apply(lambda x: (basedate - x).days)
recency_df.drop('LastOrder',axis=1,inplace=True)

#Frequency
df_temp=df_rfq.drop_duplicates(subset=['Order', 'Member'], keep="first")
#calculate frequency of orders
frequency_df = df_temp.groupby(by=['Member'], as_index=False)['Order'].count()

#Quantity
quantity_df=df_rfq.groupby(by=['Member'],as_index=False)['SKU'].count()

#Merge and rename
df_temp = recency_df.merge(frequency_df,on='Member')
df_rfq=df_temp.merge(quantity_df,on='Member')
df_rfq.rename(columns={'Order':'Frequency','SKU':'Quantity'},inplace=True)

## Modelling

In [19]:
def get_order_table(df, member, orders_since=None, orders_until=None):
    """Returns dataframe with order as rows and SKUs as columns, indicating whether SKU was bought in order or not.
    """
    if orders_until == None:
        orders_until = df.Date.max()
    if orders_since == None:
        orders_since = df.Date.min()
    df = df.sort_values("Date")
    member_df = df[(df.Member == member) & (df.Date <= orders_until) & (df.Date >= orders_since)]
    return pd.crosstab(member_df.Order, member_df.SKU)

In [20]:
def get_days_since_last_purchase(df, member, sku):
    """
    Return days since member bought sku. 
    """
    df = df.sort_values("Date")
    # this would need to be changed to datetime.now() in production environment
    now = pd.to_datetime("2014-10-07", format="%Y-%m-%d")

    return (now - df[(df.Member == member) & (df.SKU == sku)].Date.values[-1]).seconds /60 /24

In [21]:
def square_matrix(df, member, date):
    """
    Returns a square matrix with all SKU combinations of the orders of a member and the probability that they occur together.
    """
    sliced_df = df[["Order", "SKU"]][(df.Member == member)&(df.Date<=date)]
    gps = sliced_df.groupby("Order")
    unqFet = list(set(sliced_df["SKU"]))
    final = pd.DataFrame(columns=unqFet, index=unqFet)
    final = final.fillna(0)
    for g in gps.groups.values():
        for i in range(len(g)):
            for j in range(len(g)):
                if i != j:
                    final[ sliced_df["SKU"][g[i]] ][ sliced_df["SKU"][g[j]] ] += 1
    final = final.div(final.sum(),axis=0)
    finall = final.fillna(0)
    return final

In [43]:
a = square_matrix(df, "M09736", dt.datetime(2014,7,5))
a.isna().sum()

15669760    1
15669764    1
15669767    1
7753739     1
15669772    1
           ..
15668684    1
15668688    1
34990551    1
93025752    1
7593949     1
Length: 103, dtype: int64

In [23]:
def measure_similarity(basket1, basket2, distance):
    """ This function returns the similarity measure of two baskets.
    The metrics could be cosine similarity, jaccard- or dice coefficient.
    """
    if distance == "cosine":
        result = 1-spatial.distance.cosine(basket1, basket2)

    elif distance == "jaccard":
        result = 1-spatial.distance.jaccard(basket1,basket2)
        
    elif distance == "dice":
        result = 1-spatial.distance.dice(basket1,basket2)

    return result

In [24]:
recommended_items = [7543289, 15668688, 15668468, 34934495]
test_start_date = dt.datetime(2014,7,5)
member = "M64379"

def evaluate_recommendation(member, recommended_items, test_start_date):
    test_orders = get_order_table(df, member, orders_since=test_start_date)
    recommended_order = [1 if i in recommended_items else 0 for i in test_orders.columns.values]

    similarity_scores = []
    for order_id, order in test_orders.iterrows():
        score = measure_similarity(order.values, recommended_order, "jaccard")
        similarity_scores.append(score)
        return np.mean(similarity_scores)

evaluate_recommendation(member, recommended_items, test_start_date)

0.6666666666666667

In [38]:
def powerMethod(A,iter):
    n = A.shape[1]
    m = 1/n #weight of each product
    x0 = [1]*n #final vector
    delta = m*(np.array([1]*n,dtype='float64')/n) # array([1]*n is [1 1 ... 1] n times how the iteration proceed
    for i in range(iter):
        x0 = np.dot((1-m),np.dot(A,x0)) + delta
    return x0

df1_transposed = a.T # or df1.transpose()
A = np.array(df1_transposed)
powerMethod(A,200)

array([0.01497107, 0.09867281, 0.06413548, 0.20705078, 0.62082127,
       0.03951112, 0.0198125 , 0.03460667, 0.02476141, 0.0691243 ,
       0.01982056, 0.38927643, 0.01486737, 0.01980243, 0.03949506,
       0.02473994, 0.28585424, 0.08877726, 0.68005091, 0.03950857,
       0.01982076, 0.16264243, 0.07891041, 0.14787276, 0.03460667,
       0.09864804, 0.03458147, 0.11337573, 0.03949364, 0.08875913,
       0.17747233, 0.03460667, 0.12818741, 0.0986219 , 0.18727572,
       0.40899615, 0.07398627, 0.03458147, 0.03457509, 0.08382366,
       0.14787835, 0.13310364, 0.07398627, 0.01980954, 0.01490858,
       0.20698857, 0.25627027, 0.38438228, 0.64078184, 0.50261906,
       0.05427474, 0.04449518, 0.06413548, 0.82300477, 0.10845631,
       0.01497107, 0.03458607, 0.12819496, 0.46828662, 0.05920338,
       0.0296447 , 0.00994538, 0.15774299, 0.01488361, 0.00994357,
       0.01982076, 0.14301025, 0.59125158, 0.11338159, 0.03459769,
       0.05920338, 0.05928113, 0.03457654, 0.18733652, 0.31044

In [39]:
s = pd.Series(powerMethod(A,200),index=df1_transposed.columns.values)
s.sort_values(ascending=False)

15668478    0.823005
7629391     0.680051
15668688    0.660321
15668465    0.640782
15668685    0.640634
              ...   
15668454    0.014909
7587648     0.014884
15669790    0.014867
7664954     0.009945
7587651     0.009944
Length: 104, dtype: float64

In [42]:
basket1 = [15668478,7629391,15668688]
basket2 = [7587651, 7664954,15669790 ]
member = "M64379"
test_start_date = dt.datetime(2014,7,5)
evaluate_recommendation(member, basket1, test_start_date)

0.16666666666666663

In [ ]:
def get_smart_basket(df,Member,score_vector,additional_items_perc):
    """Takes the final score of function "get final score" and returns a list of recommended items of each customer based on the page rank score. The length of the list can be defined by "additional_items_perc" which is the percentage above the average number of products a customer ever bought since."""
    orders=get_order_table(df, Member, orders_since=None, orders_until=None)
    orders["counter"]=orders.sum(axis=1)
    mean_items=orders.counter.mean()
    length=round(mean_items*(1+additional_items_perc),0)
    sv=score_vector.sort_values(by=["score"])#name column according to output of final score function
    rec_list=sv.iloc[0:length,0]
    return rec_list